In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import scipy.io as sio
import os

: 

In [ ]:
## Baseline (No-human) ##
filename = "../CSI_Data_27122023/Horizontal/Q1_01.csv"

data = pd.read_csv(filename,header=None)
# Extract the 26th column
df = data.iloc[:, 25]

csi_data = []

for i in range(1,min(2000, len(df))):
    # Remove square brackets and split the string
    str1 = df[i].replace("[", "").replace("]", "")
    List = str1.split(" ")
    del List[-1]
    csi_values = [float(val) for val in List]

    # Append the list of CSI values to the csi_data list
    csi_data.append(csi_values)

# Convert csi_data to a NumPy array and then to a DataFrame
csi_data = np.array(csi_data).T.tolist()
csi_data = pd.DataFrame(csi_data)

# Display the first few rows of the DataFrame
print(csi_data)


In [ ]:
output_list = []

In [ ]:
# quartile = "Q1"
# section = "01"
quartile = "Q1"
section = "03"
sampling_range = 20

output_list = []

for r in range(1, 2):
    file_dir = "../CSI_Data_27122023/Horizontal/"
    name = quartile + "_" + section
    csv = ".csv"
    name_csv = name + csv
    file_name = os.path.join(file_dir, name_csv)
    
    # Read CSI data from the current file
    data_walk = pd.read_csv(file_name, header=None)
    df_walk = data_walk.iloc[:, 25]
    df_walk = df_walk.dropna()

    csi_data_walk = []
    
    # Iterate over a limited number of rows (e.g., 2000)
    for i in range(min(2000, len(df_walk))):
        str1 = df_walk.iloc[i].replace("[", "").replace("]", "")
        # print(str1)
        csi_values = [float(val) for val in str1.split(" ")[1:-1]]  
        csi_data_walk.append(csi_values)
    del csi_data_walk[0]
    # print(csi_data_walk)
    csi_data_walk = np.array(csi_data_walk).T.tolist()
    csi_data_walk = pd.DataFrame(csi_data_walk)
    # print("csi_data_walk\n",csi_data_walk)

    # Calculate the standard deviation for each subcarrier
    # output = []
    # for k in range(127):
    #     subcarrier_std = csi_data_walk.iloc[:, k].rolling(window=sampling_range).std()
    #     output.append(subcarrier_std)
    # output_list.append(output)
    # sd_output = pd.DataFrame(output)
    # print("output_df\n",sd_output.head())
    
    sampling_range = 20
    output = []
    constant=csi_data.mean(axis=1)
    for k in range(126):
        subcarrier = []
        for i in range(0,len(csi_data)-sampling_range):
            temp_data = []
            for j in range(i, i + sampling_range):
                temp_data.append(math.pow(csi_data_walk[j][k] - constant[k],2))
            subcarrier.append(math.sqrt(sum(temp_data)/sampling_range))
        output.append(subcarrier)
    output_list.append(output)
    sd_output = pd.DataFrame(output)
    print("output_df\n",sd_output)
    
    

: 

In [ ]:
output_array = np.array(output_list)


In [ ]:
max(sd_output),min(sd_output)

In [ ]:
plt.figure(figsize = (15, 7))
sn.heatmap(sd_output, cmap="jet", vmin=5, vmax=20)
plt.savefig(name + ".jpg")
plt.show()

In [ ]:
print(output_list)

In [ ]:
print(output_array)

In [ ]:
output_array[:15]

In [ ]:
output_reshape = output_array.reshape(output_array.shape[0], -1)
np.savetxt("csi_txrx.csv", output_reshape)

In [ ]:
load_txrx = np.loadtxt("./csi_txrx.csv")
load_rxtx = np.loadtxt("./csi_rxtx.csv")

In [ ]:
print(load_rxtx)

In [ ]:
print(load_txrx)

In [ ]:
csi_txrx= load_txrx.reshape(output_array.shape[0], output_array.shape[1], output_array.shape[2])
csi_rxtx= load_rxtx.reshape(output_array.shape[0], output_array.shape[1], output_array.shape[2])

# csi_txrx= load_txrx.reshape(output_array.shape[0], output_array.shape[1])
# csi_rxtx= load_rxtx.reshape(output_array.shape[0], output_array.shape[1])

In [ ]:
csi = np.concatenate((csi_txrx, csi_rxtx),axis=0)

In [ ]:
zeros = np.zeros(30)
ones = np.ones(30)
answer = np.concatenate((zeros, ones))


print(zeros)
print(ones)
print("answer:")
print(answer)

In [ ]:
import random
permutation = np.random.permutation(len(csi))
print(permutation)
# Use the same permutation to shuffle both arrays
shuffled_csi = csi[permutation]
shuffled_answer = answer[permutation]

print("shuffle_csi :",shuffled_csi)
print("shuffle_csi shape :",shuffled_csi.shape)

print("shuffle_answer :",shuffled_answer)
print("shuffle_answer shape :",shuffled_answer.shape)

In [ ]:
x_train = shuffled_csi[:50]
x_test = shuffled_csi[50:]
y_train = shuffled_answer[:50]
y_test = shuffled_answer[50:60]

#print data
print("x_train")
print(x_train)
print(x_train.shape)

print("x_test")
print(x_test)
print(x_test.shape)

print("y_train")
print(y_train)
print(y_train.shape)

print("y_test")
print(y_test)
print(y_test.shape)

In [ ]:
# Original
print(x_train.max())
print(x_test.max())

x_train = x_train / x_train.max()
x_test = x_test / x_test.max()
print("x_train:")
print(x_train)
print()
print("x_test:")
print(x_test)
# x_train = x_train.reshape(-1, 126, 100)
# x_test = x_test.reshape(-1, 126, 100)


In [ ]:
from tensorflow import keras
import numpy as np

# Assuming you have preprocessed input data x_train and output data y_train

# Define the model architecture
model = keras.Sequential([
    keras.layers.Input(shape=(None, 2)),  # Input shape for two streams of CSI data
    # Add layers as per your architecture design
    # Example:
    # keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu'),
    # keras.layers.MaxPooling1D(pool_size=2),
    # keras.layers.LSTM(units=64, return_sequences=True),
    # keras.layers.Dense(16, activation='relu'),
    # keras.layers.Dense(4*4, activation='softmax'),  # Output layer for 4x4 array
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(4*4, activation='sigmoid'),  # Output layer for 4x4 array
    keras.layers.Reshape((4, 4))
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary cross-entropy for binary classification
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)  # Adjust batch size and epochs as needed

# Evaluate the model if needed
# model.evaluate(x_test, y_test)


In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model.save("trained_model.h5")

In [ ]:
# OLD Code

from tensorflow import keras
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(126, 100)),
    keras.layers.Dense(8, activation='sigmoid'),
    keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)